In [1]:
"""    
    Indicador: Rendimientos del Equipo de Reconocimiento para pago
"""

'    \n    Indicador: Rendimientos del Equipo de Reconocimiento para pago\n'

<h3>Importación Librerías</h3>

In [2]:
import pandas as pd
import os
import seaborn as sns

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

import shutil

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime as dt

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias
import funcion_SQL_a_DataFrame
import funcion_parametrizacion_variables as funcion_parametrizacion_variables

<h3>Ejecución de Función de Variables</h3>

In [3]:

# TODO: Ejecución de Funciones
resultado = funcion_parametrizacion_variables.parametrizacion_variables()
consulta_tRendimientosActualizacion = resultado[2]
consulta_fechas = resultado[3]
renombrar_actividades = resultado[4]
dict_coordinador_reconocedor = resultado[5]
consulta_radicado_padre = resultado[6]

<h3>Parametrización de Variables</h3>

In [4]:
consulta_tRendimientosActualizacion = consulta_tRendimientosActualizacion

consulta_fechas = consulta_fechas

columnas_croquis = ['municipio'
    ,'radicado'
    ,'componente_responsable'
    ,'npn'
    ,'usuario_actual_responsable_tramite'
    ]

patron_busqueda = 'ransacci'

fecha_actual = dt.now()
fecha_directorio = str(fecha_actual.strftime("%Y%m%d"))

DIRECTORIO_SALIDA_XLSX = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_2_Rendimientos_Reconocimiento"
NOMBRE_XLSX = str(fecha_directorio) + "_RendimientoEquipoReconocimiento_BCGS.xlsx"
RUTA_SALIDA_XLSX = os.path.join(DIRECTORIO_SALIDA_XLSX, NOMBRE_XLSX)

DIRECTORIO_EXCEL_BASE = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\zInsumos"
NOMBRE_EXCEL_BASE = 'base_npn_analisis_seguimiento.xlsx'
RUTA_EXCEL_BASE = os.path.join(DIRECTORIO_EXCEL_BASE, NOMBRE_EXCEL_BASE)

FC_TERRENOS_UNIFICADOS = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada\TERRENO_POR_HITO"

<h3>Ejecución de Funciones</h3>

In [5]:

df_consulta_rendimientosActualizacion = funcion_SQL_a_DataFrame.sql_a_dataframe(consulta_tRendimientosActualizacion)
df_consulta_fechas = funcion_SQL_a_DataFrame.sql_a_dataframe(consulta_fechas)
df_consulta_radicado_padre = funcion_SQL_a_DataFrame.sql_a_dataframe(consulta_radicado_padre)

Inicio de la conexión
Conexión exitosa ...
Inicio de la conexión
Conexión exitosa ...
Inicio de la conexión
Conexión exitosa ...


<h3>Incorporación de Rol</h3>

In [6]:
df_consulta_rendimientosActualizacion['rol_actual_responsable_tramite'] = None
df_consulta_rendimientosActualizacion['rol_anterior_responsable_actividad'] = None
for llave, valor in dict_coordinador_reconocedor.items():
    df_consulta_rendimientosActualizacion.loc[df_consulta_rendimientosActualizacion['usuario_actual_responsable_tramite']==llave, 'rol_actual_responsable_tramite'] = valor
    df_consulta_rendimientosActualizacion.loc[df_consulta_rendimientosActualizacion['usuario_responsable_actividad_anterior']==llave, 'rol_anterior_responsable_actividad'] = valor

<h3>Cambiar nombre de actividades</h3>

In [7]:
for columna in df_consulta_rendimientosActualizacion.columns:
    if columna == 'actividad_actual_tramite':
        for llave, valor in renombrar_actividades.items():
            df_consulta_rendimientosActualizacion.loc[df_consulta_rendimientosActualizacion[columna]==llave, columna] = valor

<h3>Reporte NPN por Radicado</h3>

In [8]:
df_radicado_npn = df_consulta_rendimientosActualizacion[['municipio','componente_responsable','radicado','npn','ficha','actividad_actual_tramite','usuario_actual_responsable_tramite','con_plano_cartografico','tipo_terreno']]

# TODO: Llamadas a las variables
df_terrenos_unificados = pd.DataFrame.spatial.from_featureclass(FC_TERRENOS_UNIFICADOS)

# TODO: Se relaciona los terrenos con la tabla de relación de UI
df_terrenos_unificados = df_terrenos_unificados.rename(columns={'area_ha_cmt12':'area_ha_ctm12_predio'})
df_terrenos_unificados['area_ha_ctm12_predio'] = df_terrenos_unificados['area_ha_ctm12_predio'].round(3)

# TODO: Relación con Radicados
df_radicados_ui = pd.merge(left=df_terrenos_unificados, right=df_radicado_npn, left_on="codigo", right_on="npn", how="right")
df_radicados_ui = df_radicados_ui[['municipio', 'componente_responsable', 'radicado', 'ficha', 'npn', 'area_ha_ctm12_predio', 'actividad_actual_tramite','usuario_actual_responsable_tramite','con_plano_cartografico','tipo_terreno','id_ui','meta_hito']]

# TODO: Unificación de fechas
df_radicados_ui_fechas = pd.merge(left=df_radicados_ui, right=df_consulta_fechas, on="radicado", how="left")
df_radicados_ui_fechas_estandarizado = df_radicados_ui_fechas[['municipio', 'componente_responsable', 'radicado', 'fecha_radicado', 'ficha', 'fecha_actualizacion_tramite', 'npn', 'area_ha_ctm12_predio', 'actividad_actual_tramite','usuario_actual_responsable_tramite','con_plano_cartografico','tipo_terreno','id_ui','meta_hito']]

# TODO: Incorporación de radicados padre a reporte
df_npn_w_radicado_padre = pd.merge(left=df_radicados_ui_fechas_estandarizado,
    right=df_consulta_radicado_padre,
    on='radicado',
    how="left")

# ** Reorganización de columnas para Radicado Hijo
columnas_reorganizacion_npn = ['municipio',
    'componente_responsable',
    'radicado_padre',
    'radicado',
    'fecha_radicado',
    'fecha_actualizacion_tramite',
    'ficha',
    'npn',
    'area_ha_ctm12_predio',
    'actividad_actual_tramite',
    'usuario_actual_responsable_tramite',
    'con_plano_cartografico',
    'tipo_terreno',
    'id_ui',
    'meta_hito'
    ]

df_npn_w_radicado_padre = df_npn_w_radicado_padre[columnas_reorganizacion_npn]

# TODO: Usar ExcelWriter para guardar múltiples dataframes en diferentes hojas
with pd.ExcelWriter(RUTA_SALIDA_XLSX, engine='openpyxl') as writer:
    df_npn_w_radicado_padre.to_excel(writer, sheet_name='Radicado_X_NPN', index=False)

print(f"Se exporta XLSX {NOMBRE_XLSX}, en la ruta {DIRECTORIO_SALIDA_XLSX}")

Se exporta XLSX 20240820_RendimientoEquipoReconocimiento_BCGS.xlsx, en la ruta C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_2_Rendimientos_Reconocimiento


<h3>Indicador 2</h3>

In [9]:

# ? Indicador 2
# ** Fase Previa a la Transacción Catastral
df_rendActua_ERecono_FasePrevia = df_consulta_rendimientosActualizacion[(df_consulta_rendimientosActualizacion['fase_actividad']=='Fase Previa a la Transacción Catastral')]
df_rendActua_ERecono_indicador_2_Ordenado = pd.DataFrame(df_rendActua_ERecono_FasePrevia.groupby(['municipio','actividad_actual_tramite','rol_actual_responsable_tramite'])['usuario_actual_responsable_tramite'].value_counts()).reset_index().rename(columns={'count':'total_predios'})

# TODO: Construcción de Indicador 2
dataframes = {}
df_municipio = pd.DataFrame(df_rendActua_ERecono_indicador_2_Ordenado[['municipio']].value_counts()).reset_index()
df_municipio = df_municipio['municipio']
lista_municipio_bcgs = df_municipio.tolist()

for mpio in lista_municipio_bcgs:
    df = df_rendActua_ERecono_indicador_2_Ordenado[df_rendActua_ERecono_indicador_2_Ordenado['municipio'] == mpio]
    dataframes[mpio] = df

for dframe, df in dataframes.items():
    globals()[f'df_{dframe}_indicador_2'] = df
    # ? Generación de copia
    globals()[f'df_{dframe}_indicador_2'] = globals()[f'df_{dframe}_indicador_2'].copy()
    # ? Unificación de campos
    globals()[f'df_{dframe}_indicador_2']['rol_usuario'] = globals()[f'df_{dframe}_indicador_2']['rol_actual_responsable_tramite'] + ' - ' + globals()[f'df_{dframe}_indicador_2']['usuario_actual_responsable_tramite']
    # ? Generación Pivote
    globals()[f'df_{dframe}_indicador_2_pivote'] = globals()[f'df_{dframe}_indicador_2'].pivot(index='rol_usuario', columns='actividad_actual_tramite', values='total_predios')
    # ? Reemplazo de Nulos por 0 y cambio de float a int
    for columna in globals()[f'df_{dframe}_indicador_2_pivote'].columns:
        globals()[f'df_{dframe}_indicador_2_pivote'].loc[globals()[f'df_{dframe}_indicador_2_pivote'][columna].isnull(), columna] = 0
        globals()[f'df_{dframe}_indicador_2_pivote'][columna] = globals()[f'df_{dframe}_indicador_2_pivote'][columna].astype(int)
    # ? Reseteo de Índices
    globals()[f'df_{dframe}_indicador_2_pivote'] = globals()[f'df_{dframe}_indicador_2_pivote'].reset_index()
    # ? Impresión en XLSX
    with pd.ExcelWriter(RUTA_SALIDA_XLSX, engine='openpyxl', mode='a', if_sheet_exists='new') as writer:
       globals()[f'df_{dframe}_indicador_2_pivote'].to_excel(writer, sheet_name='Ind_2_'+dframe, index=False)

<h3>Indicador 4</h3>

In [10]:

dict_renombrado_columnas = {'radicado_y':'radicado',
    'municipio_y':'municipio',
    'usuario_responsable_actividad_anterior_y':'usuario_responsable_actividad_anterior',
    'rol_anterior_responsable_actividad_y':'rol_anterior_responsable_actividad'
    }

# ? Indicador 4
# TODO: Creación archivo base
# ! Cuidado si se actualiza la actividad por el número que se le asigna
# df_PrediosPreparandoEdicionGeo = df_consulta_rendimientosActualizacion[(df_consulta_rendimientosActualizacion['actividad_actual_tramite'] == '6.Preparando Edición Geográfica') |
#                                                                        (df_consulta_rendimientosActualizacion['actividad_actual_tramite'].str.contains(patron_busqueda))]
# df_PrediosPreparandoEdicionGeo_Base = df_PrediosPreparandoEdicionGeo[['radicado', 'municipio', 'rol_anterior_responsable_actividad', 'usuario_responsable_actividad_anterior', 'npn']]
# df_PrediosPreparandoEdicionGeo_Base = df_PrediosPreparandoEdicionGeo_Base.copy()
# df_PrediosPreparandoEdicionGeo_Base['fecha_registro'] = fecha_directorio
# df_PrediosPreparandoEdicionGeo_Base.to_excel(RUTA_EXCEL_BASE)

# TODO: Actualización del Archivo Base, partiendo del reporte XLSX creado el 17 de Junio, 2024
df_lectura_xlsx_base = pd.read_excel(RUTA_EXCEL_BASE)
df_lectura_xlsx_base = df_lectura_xlsx_base[['radicado','municipio','rol_anterior_responsable_actividad','usuario_responsable_actividad_anterior','npn','fecha_registro']]
print(f"Total de registros en archivo base: \n {df_lectura_xlsx_base.shape[0]}")

# ! Cuidado si se actualiza la actividad por el número que se le asigna
df_PrediosPreparandoEdicionGeo_Seguimiento = df_consulta_rendimientosActualizacion[(df_consulta_rendimientosActualizacion['actividad_actual_tramite'].str.contains('Preparando Edición Geográfica')) |
    (df_consulta_rendimientosActualizacion['actividad_actual_tramite'].str.contains(patron_busqueda))]

print(f"Total de registros seguimiento diario: \n {df_PrediosPreparandoEdicionGeo_Seguimiento.shape[0]} \n")

df_PrediosPreparandoEdicionGeo_Seguimiento = df_PrediosPreparandoEdicionGeo_Seguimiento[['radicado', 'municipio', 'rol_anterior_responsable_actividad', 'usuario_responsable_actividad_anterior', 'npn']]
df_cambios_base_seguimiento = pd.merge(df_lectura_xlsx_base, df_PrediosPreparandoEdicionGeo_Seguimiento, how = "right", on = "npn")
df_cambios_base_seguimiento_nuevos = df_cambios_base_seguimiento[df_cambios_base_seguimiento['radicado_x'].isnull()]
print(f"\n {df_cambios_base_seguimiento_nuevos} con {df_cambios_base_seguimiento_nuevos.shape[0]} nuevos registros \n")
if df_cambios_base_seguimiento_nuevos.shape[0] > 0:
    df_cambios_base_seguimiento_nuevos = df_cambios_base_seguimiento_nuevos[['radicado_y','municipio_y', 'rol_anterior_responsable_actividad_y', 'usuario_responsable_actividad_anterior_y','npn']]
    df_cambios_base_seguimiento_nuevos = df_cambios_base_seguimiento_nuevos.rename(columns=dict_renombrado_columnas)
    df_cambios_base_seguimiento_nuevos['fecha_registro'] = fecha_directorio
    df_cambios_base_seguimiento_unificacion = pd.concat([df_lectura_xlsx_base, df_cambios_base_seguimiento_nuevos])
    df_cambios_base_seguimiento_unificacion.to_excel(RUTA_EXCEL_BASE)
else:
    print(f"El archivo base no reporta nuevos predios en Edición Geográfica")

Total de registros en archivo base: 
 5080
Total de registros seguimiento diario: 
 1785 


 Empty DataFrame
Columns: [radicado_x, municipio_x, rol_anterior_responsable_actividad_x, usuario_responsable_actividad_anterior_x, npn, fecha_registro, radicado_y, municipio_y, rol_anterior_responsable_actividad_y, usuario_responsable_actividad_anterior_y]
Index: [] con 0 nuevos registros 

El archivo base no reporta nuevos predios en Edición Geográfica


In [11]:

df_lectura_xlsx_base = pd.read_excel(RUTA_EXCEL_BASE)

df_lectura_xlsx_base['rol_usuario'] = df_lectura_xlsx_base['rol_anterior_responsable_actividad'] + ' - ' + df_lectura_xlsx_base['usuario_responsable_actividad_anterior']
df_indicador_4 = pd.DataFrame(df_lectura_xlsx_base.groupby(['municipio','rol_usuario'])['fecha_registro'].value_counts()).reset_index().rename(columns={'count':'total_predios'})

df_indicador_4_municipio = pd.DataFrame(df_indicador_4['municipio'].value_counts()).reset_index()
df_indicador_4_municipio = df_indicador_4_municipio['municipio']
lista_municipios = df_indicador_4_municipio.tolist()

dataframes = {}
for mpio in lista_municipios:
    df = df_indicador_4[df_indicador_4['municipio']==mpio]
    dataframes[mpio] = df

for nmpio, df in dataframes.items():
    globals()[f'df_indicador_4_{nmpio}'] = df
    # ? Filtro de Columnas
    globals()[f'df_indicador_4_{nmpio}_stand'] = globals()[f'df_indicador_4_{nmpio}'][['rol_usuario','fecha_registro','total_predios']]
    # ? Construcción del Pivote
    globals()[f'df_indicador_4_{nmpio}_stand_pivote'] = globals()[f'df_indicador_4_{nmpio}_stand'].pivot(index="rol_usuario", columns="fecha_registro", values="total_predios").reset_index()
    # ? Reemplazo de valores nulos por 0
    globals()[f'df_indicador_4_{nmpio}_stand_pivote'] = globals()[f'df_indicador_4_{nmpio}_stand_pivote'].fillna(0)
    # ? Seleccionar todas las columnas numéricas
    columnas_numericas = globals()[f'df_indicador_4_{nmpio}_stand_pivote'].select_dtypes(include=['number']).columns
    # ? Convertir todas las columnas numéricas a tipo int
    globals()[f'df_indicador_4_{nmpio}_stand_pivote'][columnas_numericas] = globals()[f'df_indicador_4_{nmpio}_stand_pivote'][columnas_numericas].astype(int)
    # ? Impresión en XLSX
    with pd.ExcelWriter(RUTA_SALIDA_XLSX, engine='openpyxl', mode='a', if_sheet_exists='new') as writer:
        globals()[f'df_indicador_4_{nmpio}_stand_pivote'].to_excel(writer, sheet_name='Indicador_4_'+nmpio, index=False)
        print(globals()[f'df_indicador_4_{nmpio}_stand_pivote'])

fecha_registro                       rol_usuario  20240618  20240619  \
0                    Reconocedor - angel.banquez        76         0   
1                   Reconocedor - celci.quintana         0         0   
2                  Reconocedor - emilio.castillo         0         0   
3                   Reconocedor - fabio.barragan         0         0   
4                   Reconocedor - german.panizza         0         0   
5                Reconocedor - gustavo.maldonado         0         0   
6                Reconocedor - harlingto.padilla         0         0   
7               Reconocedor - hernando.sarmiento         9         0   
8                   Reconocedor - jarod.macareno         0        17   
9                    Reconocedor - karen.escobar         0         0   
10                    Reconocedor - luis.jimenez         0         0   
11                 Reconocedor - maria.hernandez        42         0   
12                   Reconocedor - mary.conquett        27      